### Data collection

Grab recent posts from r/leafs the Toronto Maple Leafs subreddit

Top 5 posts from the past week

In [12]:
import praw
import os
from dotenv import load_dotenv

load_dotenv()

reddit = praw.Reddit(client_id=os.environ.get("REDDIT_CLIENT_ID"),
                     client_secret=os.environ.get("REDDIT_CLIENT_SECRET"),
                     user_agent=os.environ.get("REDDIT_USER_AGENT"))

subreddit = reddit.subreddit('leafs')
top_posts = subreddit.top(time_filter='week', limit=5)

posts = []
for post in top_posts:
    posts.append(post)

content_list = []
title_list = []
comments_list = []
for post in posts:
    title_list.append(post.title) 
    content_list.append(f"{post.title} {post.selftext}"[:128]) 

    submission = reddit.submission(id=post.id)
    submission.comments.replace_more(limit=None) 
    
    for comment in submission.comments.list():
        if len(comments_list) >= 100:  
            break
        comments_list.append(comment.body)
        content_list.append(comment.body)

print(f"Total Comments Collected: {len(comments_list)}")
for idx, comment in enumerate(comments_list[:10]):  
    print(f"{idx+1}: {comment}")

Total Comments Collected: 100
1: We need this video preserved for the rest of time.
2: This makes losing to the Panthers and the Sens better.
3: This is the kind of reporting I come here for!
4: New Dart Guy Confirmed... get this chick to the playoffs
5: thank-you sportstits
6: The Scottish are such fucking beauties. Great humans.
7: This adds up. Big game tomorrow.
8: Get her playoff tickets. We found our new Dart Guy and it’s Scottish Lady.
9: Need this type of post game analysis after every game
10: I'll pass out with me head on those massive tits! HELL YEAH


### Sentiment analysis

In [13]:
from transformers import pipeline

os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

sentiment_analyzer = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    top_k=2
)

sentiments_list = []
for text in content_list:
    sentiments = sentiment_analyzer(text)
    
    sentiments_list.append(sentiments[0])

    print(f"Text: {text}")
    for sentiment in sentiments[0]:
        print(f"- {sentiment['label']}: {sentiment['score']:.2f}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Text: Why the Leafs lost their last 2 
- negative: 0.60
- neutral: 0.39
Text: We need this video preserved for the rest of time.
- neutral: 0.62
- positive: 0.20
Text: This makes losing to the Panthers and the Sens better.
- positive: 0.51
- neutral: 0.37
Text: This is the kind of reporting I come here for!
- positive: 0.89
- neutral: 0.10
Text: New Dart Guy Confirmed... get this chick to the playoffs
- positive: 0.58
- neutral: 0.42
Text: thank-you sportstits
- positive: 0.92
- neutral: 0.07
Text: The Scottish are such fucking beauties. Great humans.
- positive: 0.93
- negative: 0.03
Text: This adds up. Big game tomorrow.
- positive: 0.78
- neutral: 0.22
Text: Get her playoff tickets. We found our new Dart Guy and it’s Scottish Lady.
- positive: 0.54
- neutral: 0.44
Text: Need this type of post game analysis after every game
- neutral: 0.75
- positive: 0.15
Text: I'll pass out with me head on those massive tits! HELL YEAH
- positive: 0.69
- neutral: 0.16
Text: I want to drink with thi

### Summarise using ChatGPT

In [14]:
import openai
from IPython.display import display, Markdown, clear_output
import time

openai.api_key = os.environ.get("CHATGPT_API_KEY")

system_prompt = """
    You are ice hockey analyst. Your goal is to determine how confident the Toronto Maple Leafs fans are about the team winning the cup this year.

    You will provide a score based on the analysed sentiment scores combined with your own analysis rating the fans confidence in starting "parade planning" for when they win the cup.
    You will also summarise the key points of confidence and concern the fans have with the team.

    Your response should be accurate and formatted using markdown using the following outline:
        1. Confidence score (The parade planning metre)
        2. Points of confidence (between 2-5 points)
        3. Points of concern (between 2-5 points)
        4. A short paragraph summarising the output.
"""

user_prompt = """
    These are the most recent comments and posts along with their pre-analyzed sentiment scores made on the leafs subreddit.
"""
for index, content in enumerate(content_list):
    user_prompt = user_prompt + f"""
        Text: {content}
        Sentiments: 
            1. {sentiments_list[index][0]['label']}: {sentiments_list[index][0]['score']:.2f}
            2. {sentiments_list[index][1]['label']}: {sentiments_list[index][1]['score']:.2f}
    """

stream = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    stream=True
)

full_response = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        content = chunk.choices[0].delta.content
        full_response += content
        
        clear_output(wait=True)
        display(Markdown(full_response))
        
        time.sleep(0.01)

1. Confidence Score (The Parade Planning Metre)
Parade Planning Metre: 7/10 🎉
Fans are showing notable enthusiasm and high spirits, driven largely by the recent viral sensation of the "Scottish Lady" bringing joy and humor to the fanbase. While confidence is running high, a few concerns about inconsistent performance and recent losses temper the excitement.

2. Points of Confidence 🚀
Positive Vibes and Entertainment Factor: The viral “Scottish Lady” has ignited a wave of fun, camaraderie, and positive sentiment, adding a spark to the fanbase. Comments about her being a “legend” and a “masterpiece” are dominating the conversation.
Playoff Mindset Developing: Some comments indicate optimism about the team "finding their playoff stride" and being "reset for Calgary," showing belief that the Leafs can bounce back when it matters.
Fan Engagement and Energy: There’s a palpable sense of excitement, with calls to get the viral sensation involved in the playoffs and humor driving engagement. This collective hype is translating into elevated morale.
Belief in Cup Potential: A standout comment—"Cup win confirmed"—shows that some fans still hold strong hope that this is the year.
Memorable Moments Building Fan Optimism: The hilarity and uniqueness of the viral content have given fans a distraction from past disappointments, making the journey more enjoyable.
3. Points of Concern 😬
Inconsistent Performance: Mentions of the Leafs losing to teams like the Panthers and Senators have fans worried about the team’s ability to maintain a winning streak.
Defensive Concerns: Some posts subtly imply worries about defensive lapses, leading to a lack of confidence in maintaining leads during critical games.
Past Playoff Disappointments: There’s an underlying feeling of caution, with some fans hesitant to fully embrace optimism due to the Leafs’ historical struggles in the playoffs.
Focus and Distractions: Some jokes and comments suggest that the off-ice distractions (like the Scottish Lady) could be fun but potentially divert attention from the team’s playoff mindset.
Lingering Negativity: A handful of comments express frustration or concern about the team’s ability to stay consistent and prepared for critical games.
4. Summary 📝
Leafs Nation is buzzing with positive energy, driven by the viral phenomenon of the "Scottish Lady" and the humor that has captured the hearts of fans. The excitement is high, with many jokingly calling for her to be involved in the playoff push and believing she’s a good luck charm. However, concerns remain about inconsistent performance and the team’s ability to maintain focus as they approach the playoffs. While fans are not fully planning the parade yet, confidence is definitely on the rise, and the momentum could swing in the Leafs’ favor with a few solid performances.